# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [3]:
document = ET.parse( './data/mondial_database.xml' )

#### 10 countries with the lowest infant mortality rates

In [10]:
infant_mortality = {}
for root in document.iterfind('country'):
    for subelement in root.iterfind('infant_mortality'):
        infant_mortality[root.find('name').text] = float(subelement.text)

In [11]:
infant_mortality

{'Afghanistan': 117.23,
 'Albania': 13.19,
 'Algeria': 21.76,
 'American Samoa': 8.92,
 'Andorra': 3.69,
 'Angola': 79.99,
 'Anguilla': 3.4,
 'Antigua and Barbuda': 13.29,
 'Argentina': 9.96,
 'Armenia': 13.97,
 'Aruba': 11.74,
 'Australia': 4.43,
 'Austria': 4.16,
 'Azerbaijan': 26.67,
 'Bahamas': 12.5,
 'Bahrain': 9.68,
 'Bangladesh': 45.67,
 'Barbados': 10.93,
 'Belarus': 3.64,
 'Belgium': 4.18,
 'Belize': 20.31,
 'Benin': 57.09,
 'Bermuda': 2.48,
 'Bhutan': 37.89,
 'Bolivia': 38.61,
 'Bosnia and Herzegovina': 5.84,
 'Botswana': 9.38,
 'Brazil': 19.21,
 'British Virgin Islands': 13.45,
 'Brunei': 10.48,
 'Bulgaria': 15.08,
 'Burkina Faso': 76.8,
 'Burundi': 63.44,
 'Cambodia': 51.36,
 'Cameroon': 55.1,
 'Canada': 4.71,
 'Cape Verde': 24.28,
 'Cayman Islands': 6.21,
 'Central African Republic': 92.86,
 'Chad': 90.3,
 'Chile': 7.02,
 'China': 14.79,
 'Colombia': 15.02,
 'Comoros': 65.31,
 'Congo': 59.34,
 'Cook Islands': 14.33,
 'Costa Rica': 8.7,
 'Cote dIvoire': 60.16,
 'Croatia': 5

In [12]:
import pandas as pd

In [14]:
infantmortality_df = pd.DataFrame.from_dict(infant_mortality, orient = 'index')

In [15]:
infantmortality_df.columns = ['infant_mortality']

In [16]:
infantmortality_df.index.names = ['country']

In [17]:
infantmortality_df.head()

,infant_mortality
country,
Canada,4.71
Sao Tome and Principe,49.16
Turkmenistan,38.13
Saint Helena,17.63
Lithuania,6.00


In [22]:
infantmortality_df.sort_values(by = 'infant_mortality', ascending = True). head(10)

,infant_mortality
country,
Monaco,1.81
Japan,2.13
Norway,2.48
Bermuda,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13


#### 10 cities with the largest population

In [23]:
current_pop = 0
current_pop_year = 0
citypop = {}

for country in document.iterfind('country'):
    for city in country.getiterator('city'):
        for subelement in city.iterfind('population'):
            if int(subelement.attrib['year']) > current_pop_year:
                current_pop = int(subelement.text)
                current_pop_year = int(subelement.attrib['year'])
        citypop[city.findtext('name')] = current_pop
        current_pop = 0
        current_pop_year = 0

In [24]:
citypop_df = pd.DataFrame.from_dict(citypop, orient = 'index')

In [25]:
citypop_df.columns = ['population']

In [26]:
citypop_df.index.names = ['city']

In [27]:
citypop_df.sort_values(by='population', ascending=False).head(10)

,population
city,
Shanghai,22315474
Istanbul,13710512
Mumbai,12442373
Moskva,11979529
Beijing,11716620
São Paulo,11152344
Tianjin,11090314
Guangzhou,11071424
Delhi,11034555


#### 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [30]:
eg = {}
current_pop = 0
current_pop_year = 0

In [37]:
for country in document.iterfind('country'):
    for population in country.getiterator('population'):
        if int(population.attrib['year']) > current_pop_year:
            current_pop = int(population.text)
            current_pop_year = int(population.attrib['year'])
    for ethn_gp in country.iterfind('ethnicgroup'):
        if ethn_gp.text in eg:
            eg[ethn_gp.text] += current_pop*float(ethn_gp.attrib['percentage'])/100
        else:
            eg[ethn_gp.text] = current_pop*float(ethn_gp.attrib['percentage'])/100        
    current_pop = 0
    current_pop_year = 0

In [38]:
ethnic_df = pd.DataFrame.from_dict(eg, orient='index')

In [39]:
ethnic_df.columns = ['population']
ethnic_df.index.names = ['ethnic_group']
ethnic_df.groupby(ethnic_df.index).sum().sort_values(by='population', ascending=False).head(10)

,population
ethnic_group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.304840e+08
Japanese,1.265342e+08


#### name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [55]:
river_country=None
river_name=None
river_length=0
lake_country=None
lake_name=None
lake_area=0
airport_country=None
airport_name=None
airport_elv=0

In [56]:
for river in document.iterfind('river'):
    for length in river.iterfind('length'):
        if river_length < float(length.text):
            river_length = float(length.text)
            river_country = river.attrib['country']
            river_name = river.findtext('name')

In [57]:
for lake in document.iterfind('lake'):
    for area in lake.iterfind('area'):
        if lake_area < float(area.text):
            lake_area = float(area.text)
            lake_country = lake.attrib['country']
            lake_name = lake.findtext('name')

In [58]:
for airport in document.iterfind('airport'):
    for elv in airport.iterfind('elevation'):
        if (elv.text is not None) and (airport_elv < float(elv.text)):
            airport_elv = float(elv.text)
            airport_country = airport.attrib['country']
            airport_name = airport.findtext('name')

In [59]:
data = [[lake_name, river_name, airport_name], [lake_country, river_country, airport_country], [lake_area, river_length, airport_elv]]

In [60]:
df = pd.DataFrame(data, columns = ['Largest Lake', 'Longest River', 'Highest Airport'], index=['Name', 'Location (Country Code)', 'Metric Value'])

In [61]:
df

,Largest Lake,Longest River,Highest Airport
Name,Caspian Sea,Amazonas,El Alto Intl
Location (Country Code),R AZ KAZ IR TM,CO BR PE,BOL
Metric Value,386400,6448,4063
